In [4]:
import pandas as pd
import numpy as np
import re
import joblib
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [5]:
def get_features(text):
    features = {}
    features['len'] = len(text)
    features['num_upper'] = sum(map(str.isupper, text))
    features['num_digits'] = sum(map(str.isdigit, text))
    features['num_excl'] = text.count('!')
    features['num_url'] = len(re.findall(r'http[s]?://\\S+', text))
    features['num_stopwords'] = sum(1 for w in text.lower().split() if w in stopwords.words('english'))
    features['word_count'] = len(text.split())
    features['mean_word'] = np.mean([len(w) for w in text.split()]) if text.split() else 0
    return features

In [6]:
print("Загружаем данные...")
data_fake = pd.read_csv('input_data/Fake.csv')
data_true = pd.read_csv('input_data/True.csv')
data_fake['label'] = 0
data_true['label'] = 1
df = pd.concat([data_fake, data_true], axis=0)

Загружаем данные...


In [7]:
print("Создание признаков...")
feature_matrix = pd.DataFrame([get_features(x) for x in tqdm(df['text'], desc="feature engineering")])
tfidf = TfidfVectorizer(max_features=500)

Создание признаков...


feature engineering: 100%|████████████████| 44898/44898 [10:30<00:00, 71.16it/s]


In [16]:
print("Вычисление TF-IDF...")
tfidf_matrix = tfidf.fit_transform(tqdm(df['text'], desc="tfidf")).toarray()
X_full = np.hstack([tfidf_matrix, feature_matrix.values])
Y = df['label'].values

Вычисление TF-IDF...


tfidf: 100%|████████████████████████████| 44898/44898 [00:04<00:00, 9177.15it/s]


In [18]:
print("Масштабирование признаков...")
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_full_scaled = scaler.fit_transform(X_full)
X_train, X_test, y_train, y_test = train_test_split(X_full_scaled, Y, test_size=0.2, random_state=42)


Масштабирование признаков...


In [19]:
print("Обучение ансамбля моделей...")

clf_rf = RandomForestClassifier(n_estimators=120, warm_start=True, random_state=42)
rf_estimators_step = 20
rf_total = 120
for n in tqdm(range(rf_estimators_step, rf_total+1, rf_estimators_step), desc="RandomForest"):
    clf_rf.n_estimators = n
    clf_rf.fit(X_train, y_train)


Обучение ансамбля моделей...


RandomForest: 100%|███████████████████████████████| 6/6 [00:19<00:00,  3.18s/it]


In [20]:
print("Обучаем LogisticRegression (это может занять время)...")
clf_lr = LogisticRegression(max_iter=10000, solver='saga')
clf_lr.fit(X_train, y_train)
print("LogisticRegression обучен.")

Обучаем LogisticRegression (это может занять время)...
LogisticRegression обучен.


In [21]:
ensemble = VotingClassifier(estimators=[
    ('lr', clf_lr),
    ('rf', clf_rf)
], voting='soft')
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=10000,
                                                 solver='saga')),
                             ('rf',
                              RandomForestClassifier(n_estimators=120,
                                                     random_state=42,
                                                     warm_start=True))],
                 voting='soft')

In [22]:
print("Сохраняем модель и параметры...")
joblib.dump(ensemble, "models/model_rf_lr_voting.pkl")
joblib.dump(tfidf, "models/tfidf.pkl")
joblib.dump(scaler, "models/scaler.pkl")
joblib.dump(feature_matrix.columns.tolist(), "models/feature_names.pkl")

print("Модель, TF-IDF, scaler и список признаков сохранены!")

Сохраняем модель и параметры...
Модель, TF-IDF, scaler и список признаков сохранены!
